In [15]:
import os
import faiss
import numpy as np
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.notebook import tqdm
import pyarrow as pa
import pyarrow.parquet as pq
import sys
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('.'))))  # /src

from semantic_search.misc import LocalEmbeddingModel, FAISSDocumentStore

## Create vector DB

In [1]:
from importlib import reload
from semantic_search import misc
reload(misc)
from semantic_search.misc import LocalEmbeddingModel, FAISSDocumentStore

/cluster/home/lcarretero/python_envs/dsl-research-assistant/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [27]:
embedding_model = LocalEmbeddingModel(
    model_name='sentence-transformers/all-MiniLM-L6-v2',
    device='cuda',
    batch_size=8
)
document_store = FAISSDocumentStore(
    embedding_model=embedding_model,
    db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/chunk1-txt-all',
)

if not document_store.load_index():
    document_store.create_index('/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt')

Loaded index with 73809 vectors


In [26]:
print(document_store.search("visual", top_k=5)[0]['chunk_text'])

er-efficient visual instruction model. arXiv 2304.15010 , 2023. 1, 3, 4
- [9] Yash Goyal, Tejas Khot, Douglas Summers-Stay, Dhruv Batra, and Devi Parikh. Making the V in VQA matter: Elevating the role...


## Create metadata for CVPR dataset

In [53]:
# Set pandas display options to show wider dataframes
pd.set_option('display.max_colwidth', None)  # Show full text in columns
pd.set_option('display.width', 1000)         # Set the display width
pd.set_option('display.max_columns', 20)     # Show more columns

In [49]:
import re
import pyalex
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from functools import partial

raw_dir = '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt'

df = pd.DataFrame([(str(fpath), fpath.name) for fpath in Path(raw_dir).glob("*.txt")], columns=['fpath', 'fname'])

def get_title(fpath: str):
    doc_text = Path(fpath).read_text(encoding="utf-8")
    title_match = re.search(r'## ([^\n#]+)', doc_text)
    return title_match.group(1) if title_match else None

def get_metadata(title: str):
    search_results = pyalex.Works().search(title).select(['id', 'doi', 'referenced_works']).get(page=1, per_page=1)
    return (search_results[0]['doi'], search_results[0]['id'], search_results[0]['referenced_works']) if search_results else (None, None, None)

df['title'] = df['fpath'].apply(get_title)

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(tqdm(
        executor.map(get_metadata, df['title'].values),
        total=len(df)
    ))
df['doi', 'oaid', 'referenced_works'] = results

  0%|          | 0/1142 [00:00<?, ?it/s]

,fpath,fname,title,doi,oaid
0,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Bai_Fixed_Point_Diffusion_Models_CVPR_2024_paper.txt,Bai_Fixed_Point_Diffusion_Models_CVPR_2024_paper.txt,Fixed Point Diffusion Models,https://doi.org/10.1063/1.2121687,https://openalex.org/W2000456051
1,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Li_BEVNeXt_Reviving_Dense_BEV_Frameworks_for_3D_Object_Detection_CVPR_2024_paper.txt,Li_BEVNeXt_Reviving_Dense_BEV_Frameworks_for_3D_Object_Detection_CVPR_2024_paper.txt,BEVNeXt: Reviving Dense BEV Frameworks for 3D Object Detection,https://doi.org/10.1109/cvpr52733.2024.01901,https://openalex.org/W4402727763
2,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Savchenko_Leveraging_Pre-trained_Multi-task_Deep_Models_for_Trustworthy_Facial_Analysis_in_CVPRW_2024_paper.txt,Savchenko_Leveraging_Pre-trained_Multi-task_Deep_Models_for_Trustworthy_Facial_Analysis_in_CVPRW_2024_paper.txt,Leveraging Pre-trained Multi-task Deep Models for Trustworthy Facial Analysis in Affective Behaviour Analysis in-the-Wild,https://doi.org/10.1109/cvprw63382.2024.00473,https://openalex.org/W4402916217


In [94]:
df = pd.concat([df, pd.DataFrame(results, columns=['doi', 'oaid', 'referenced_works'])], axis=1)

### Store/Load paper metadata

In [12]:
# df.to_csv('/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev/paper-metadata.csv', index=False)
df = pd.read_csv('/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev/paper-metadata.csv')

def parse_referenced_works(x):
    if pd.isna(x) or x is None:
        return []
    elif isinstance(x, str):
        # Remove brackets and split by commas, then strip quotes and whitespace
        if x.startswith('[') and x.endswith(']'):
            # Example: "['https://openalex.org/W10789807', 'https://openalex.org/W109508954']"
            items = x[1:-1].split(',')
            return [item.strip().strip("'\"") for item in items if item.strip()]
        return [x]  # If it's a string but not a list format, treat as single item
    return []

df['referenced_works'] = df['referenced_works'].apply(parse_referenced_works)


### Calculate total references and references within dataset

In [13]:
# Count total references and references within the dataset
def count_references(row):
    total_refs = len(row['referenced_works'])
    dataset_oaids = set(df['oaid'].dropna())
    refs_in_dataset = sum(1 for ref in row['referenced_works'] if ref in dataset_oaids)
    return total_refs, refs_in_dataset

ref_counts = df.apply(count_references, axis=1, result_type='expand')
df['total_references'] = ref_counts[0]
df['references_in_dataset'] = ref_counts[1]


print(f"Total references stats: min={df['total_references'].min()}, max={df['total_references'].max()}, mean={df['total_references'].mean():.2f}")
print(f"References in dataset stats: min={df['references_in_dataset'].min()}, max={df['references_in_dataset'].max()}, mean={df['references_in_dataset'].mean():.2f}")


Total references stats: min=0, max=621, mean=27.32
References in dataset stats: min=0, max=3, mean=0.21


In [6]:
df.head(n=10)

,fpath,title,doi,oaid,referenced_works,total_references,references_in_dataset
0,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Fixed Point Diffusion Models,https://doi.org/10.1063/1.2121687,https://openalex.org/W2000456051,"[https://openalex.org/W1504980292, https://ope...",55,0
1,/cluster/home/lcarretero/workspace/dsl/dsl-res...,BEVNeXt: Reviving Dense BEV Frameworks for 3D ...,https://doi.org/10.1109/cvpr52733.2024.01901,https://openalex.org/W4402727763,"[https://openalex.org/W1861492603, https://ope...",73,0
2,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Leveraging Pre-trained Multi-task Deep Models ...,https://doi.org/10.1109/cvprw63382.2024.00473,https://openalex.org/W4402916217,[],0,0
3,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Insights from the Use of Previously Unseen Neu...,https://doi.org/10.48550/arxiv.2404.02189,https://openalex.org/W4393967825,[],0,0
4,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Efficient local correlation volume for unsuper...,https://doi.org/10.1109/cvprw63382.2024.00049,https://openalex.org/W4402904316,"[https://openalex.org/W1513100184, https://ope...",28,0
5,/cluster/home/lcarretero/workspace/dsl/dsl-res...,uTRAND: Unsupervised Anomaly Detection in Traf...,https://doi.org/10.1109/cvprw63382.2024.00759,https://openalex.org/W4402904312,[],0,0
6,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Test-Time Adaptation with SaLIP: A Cascade of ...,https://doi.org/10.1109/cvprw63382.2024.00526,https://openalex.org/W4402916510,[],0,0
7,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Towards Understanding and Improving Adversaria...,https://doi.org/10.1109/cvpr52733.2024.02336,https://openalex.org/W4402753640,"[https://openalex.org/W2517229335, https://ope...",31,0
8,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Adaptive Softassign via Hadamard-Equipped Sink...,https://doi.org/10.1109/cvpr52733.2024.01670,https://openalex.org/W4402753930,"[https://openalex.org/W1587878450, https://ope...",42,0
9,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Augmented Self-Mask Attention Transformer for ...,https://doi.org/10.1109/cvprw63382.2024.00705,https://openalex.org/W4402904140,"[https://openalex.org/W2507009361, https://ope...",25,1


### Misc

In [5]:
import re

raw_dir = '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt'

def check_single_doc(fpath: str):
    doc_text = fpath.read_text(encoding="utf-8")
    abstract_match = re.search(r'## Abstract\n\n(.*?)(?=\n\n## \d+\.)', doc_text, re.DOTALL)
    return abstract_match.group(1) if abstract_match else None

for fpath in Path(raw_dir).glob("*.txt"):
    if check_single_doc(fpath) is None:
        print(fpath)

# abstracts =[check_single_doc(fpath) for fpath in Path(raw_dir).glob("*.txt")]

/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Korycki_Class-Incremental_Mixture_of_Gaussians_for_Deep_Continual_Learning_CVPRW_2024_paper.txt
/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Hoque_IrrNet_Spatio-Temporal_Segmentation_Guided_Classification_for_Irrigation_Mapping_CVPRW_2024_paper.txt
/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Zhang_MOHO_Learning_Single-view_Hand-held_Object_Reconstruction_with_Multi-view_Occlusion-Aware_Supervision_CVPR_2024_paper.txt
/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Paissan_Structured_Sparse_Back-propagation_for_Lightweight_On-Device_Continual_Learning_on_Microcontroller_CVPRW_2024_paper.txt
/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Halawa_Multi-Task_Multi-Modal_Self-Super

In [4]:
pd.Series(str(type(a)) for a in abstracts).value_counts()

<class 'str'>         1133
<class 'NoneType'>       9
Name: count, dtype: int64

In [21]:
# Example search function
def search_documents(query, top_k=5):
    results = document_store.search(query, top_k=top_k)
    print(f"Search results for: '{query}'")
    for result in results:
        print(f"Rank {result['rank']} (Score: {result['score']:.4f})")
        print(f"Document: {result['document_name']}")
        print(f"Preview: {result['chunk_text']}")
        print("-" * 80)
    return results

# Test the search
search_results = search_documents("visual odometry in robotics applications", top_k=3)


Search results for: 'visual odometry in robotics applications'
Rank 1 (Score: 0.7618)
Document: Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt
Preview: omain (real data) while achieving state-of-the-art performance on the KITTI dataset.

## 1. Introduction

Visual odometry (VO) is a crucial aspect of robotics that enables machines to measure the ego-...
--------------------------------------------------------------------------------
Rank 2 (Score: 0.7586)
Document: Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt
Preview: current neural network for (un-) supervised learning of monocular video visual odometry and depth. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition , pages 5555-556...
--------------------------------------------------------------------------------
Rank 3 (Score: 0.7444)
Document: Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt
Preview: This CVPR Worksh

In [ ]:
document_store.search("visual odometry in robotics applications"*100000, top_k=3)

[{'rank': 1,
  'score': np.float32(0.506165),
  'document_id': 0,
  'document_name': 'Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt',
  'document_path': 'example_data/Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt',
  'chunk_text': 'omain (real data) while achieving state-of-the-art performance on the KITTI dataset.\n\n## 1. Introduction\n\nVisual odometry (VO) is a crucial aspect of robotics that enables machines to measure the ego-...'},
 {'rank': 2,
  'score': np.float32(0.47257516),
  'document_id': 0,
  'document_name': 'Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt',
  'document_path': 'example_data/Abouee_Weakly_Supervised_End2End_Deep_Visual_Odometry_CVPRW_2024_paper.txt',
  'chunk_text': 'ons on robotics , 33(5):1255-1262, 2017. 2, 6\n- [14] David Nist´ er, Oleg Naroditsky, and James Bergen. Visual\n\n- odometry. In Proceedings of the 2004 IEEE Computer Society Conference on Computer Visi...'

## Testing SemanticScholar API

In [14]:
# First, import the client from semanticscholar module
from semanticscholar import SemanticScholar

# You'll need an instance of the client to request data from the API
sch = SemanticScholar()

# Get a paper by its ID
paper = sch.get_paper('10.1093/mind/lix.236.433')

# Print the paper title
print(paper.title)

Computing Machinery and Intelligence


In [36]:
(df.total_references > 0).sum()/len(df)

0.45271453590192645

In [47]:
raw_dir = '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt'

df = pd.DataFrame([(str(fpath), fpath.name) for fpath in Path(raw_dir).glob("*.txt")], columns=['fpath', 'fname'])

In [60]:
import re

def get_title(fpath: str):
    doc_text = Path(fpath).read_text(encoding="utf-8")
    title_match = re.search(r'## ([^\n#]+)', doc_text)
    if title_match is None:  # Fallback: Extract from file name
        return ' '.join(Path(fpath).stem.split('_')[1:-3])
    return title_match.group(1)

df['title'] = df['fpath'].apply(get_title)

In [ ]:
from semanticscholar import SemanticScholar
from semanticscholar.SemanticScholarException import ObjectNotFoundException
from concurrent.futures import ThreadPoolExecutor

sch = SemanticScholar()

def get_paper_metadata(paper_title: str):
    try:
        paper = sch.search_paper(query=paper_title, match_title=True, fields=['paperId', 'externalIds', 'abstract'])
        res = {'paperId': paper['paperId'], 'abstract': paper['abstract']}
        res.update({f'externalIds.{k}': v for k, v in paper['externalIds'].items()})
        return res
    except ObjectNotFoundException:
        return {}

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(tqdm(
        executor.map(get_paper_metadata, df['title'].values[:3]),
        total=len(df)
    ))

  0%|          | 0/1142 [00:00<?, ?it/s]

In [72]:
df2 = pd.concat([df, pd.DataFrame(results)], axis=1)

In [73]:
df2.head()

,fpath,fname,title,paperId,abstract,externalIds.ArXiv,externalIds.DOI,externalIds.CorpusId,externalIds.DBLP
0,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Bai_Fixed_Point_Diffusion_Models_CVPR_2024_paper.txt,Bai_Fixed_Point_Diffusion_Models_CVPR_2024_paper.txt,Fixed Point Diffusion Models,fdb679246a2125dad1628081e45efb7a1c80f2c7,"We introduce the Fixed Point Diffusion Model (FPDM), a novel approach to image generation that integrates the concept of fixed point solving into the framework of diffusion-based generative modeling. Our approach embeds an im-plicit fixed point solving layer into the denoising network of a diffusion model, transforming the diffusion process into a sequence of closely-related fixed point problems. Combined with a new stochastic training method, this approach significantly reduces model size, reduces memory usage, and accelerates training. Moreover, it enables the development of two new techniques to improve sampling efficiency: reallocating computation across timesteps and reusing fixed point solutions between timesteps. We con-duct extensive experiments with state-of-the-art models on ImageNet, FFHQ, CelebA-HQ, and LSUN-Church, demon-strating substantial improvements in performance and effi-ciency. Compared to the state-of-the-art DiT model [38], FPDM contains 87% fewer parameters, consumes 60% less memory during training, and improves image generation quality in situations where sampling computation or time is limited. Our code and pretrained models are available at https://lukemelas.github.io/fixed-point-diffusion-models/.",2401.08741,10.1109/CVPR52733.2024.00901,267027739.0,NaN
1,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Li_BEVNeXt_Reviving_Dense_BEV_Frameworks_for_3D_Object_Detection_CVPR_2024_paper.txt,Li_BEVNeXt_Reviving_Dense_BEV_Frameworks_for_3D_Object_Detection_CVPR_2024_paper.txt,BEVNeXt: Reviving Dense BEV Frameworks for 3D Object Detection,a23f6180d6908499a8238c06f4fb57bf431a2b43,"Recently, the rise of query-based Transformer decoders is reshaping camera-based 3D object detection. These query-based decoders are surpassing the traditional dense BEV (Bird's Eye View)-based methods. However, we argue that dense BEV frameworks remain important due to their out-standing abilities in depth estimation and object localization, depicting 3D scenes accurately and comprehensively. This paper aims to address the drawbacks of the existing dense BEV-based 3D object detectors by introducing our proposed enhanced components, including a CRF-modulated depth estimation module enforcing object-level consistencies, a long-term temporal aggregation module with extended receptive fields, and a two-stage object decoder combining perspective techniques with CRF-modulated depth embedding. These enhancements lead to a “modernized” dense BEV framework dubbed BEVNeXt. On the nuScenes benchmark, BEVNeXt outperforms both BEV-based and query-based frameworks under various settings, achieving a state-of-the-art result of 64.2 NDS on the nuScenes test set.",2312.01696,10.1109/CVPR52733.2024.01901,265609098.0,journals/corr/abs-2312-01696
2,/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/Conversions/opencvf-data/txt/Savchenko_Leveraging_Pre-trained_Multi-task_Deep_Models_for_Trustworthy_Facial_Analysis_in_CVPRW_2024_paper.txt,Savchenko_Leveraging_Pre-trained_Multi-task_Deep_Models_for_Trustworthy_Facial_Analysis_in_CVPRW_2024_paper.txt,Leveraging Pre-trained Multi-task Deep Models for Trustworthy Facial Analysis in Affective Behaviour Analysis in-the-Wild,9c50996ba35eb605cb9bcd5835103b441cf38e07,"This article presents our results for the sixth Affective Behavior Analysis in-the-wild (ABAW) competition. To improve the trustworthiness of facial analysis, we study the possibility of using pre-trained deep models that extract reliable emotional features without the need to fine-tune the neural

### What to retrieve:
Database papers:
- paperId, externalIds, abstract, referenceCount

Then get references for each of those papers:
- paperId, externalIds, abstract

dict_keys(['paperId', 'externalIds', 'title', 'abstract', 'openAccessPdf', 'authors'])

In [50]:
res.items[0]['citedPaper']

{'paperId': '9b91b3031ea159e4964d18b2ce703168660ecf46',
 'externalIds': {'ArXiv': '2401.11605',
  'DBLP': 'journals/corr/abs-2401-11605',
  'DOI': '10.48550/arXiv.2401.11605',
  'CorpusId': 267069338},
 'corpusId': 267069338,
 'publicationVenue': {'id': 'fc0a208c-acb7-47dc-a0d4-af8190e21d29',
  'name': 'International Conference on Machine Learning',
  'type': 'conference',
  'alternate_names': ['ICML', 'Int Conf Mach Learn'],
  'url': 'https://icml.cc/'},
 'url': 'https://www.semanticscholar.org/paper/9b91b3031ea159e4964d18b2ce703168660ecf46',
 'title': 'Scalable High-Resolution Pixel-Space Image Synthesis with Hourglass Diffusion Transformers',
 'abstract': None,
 'venue': 'International Conference on Machine Learning',
 'year': 2024,
 'referenceCount': 60,
 'citationCount': 38,
 'influentialCitationCount': 7,
 'isOpenAccess': False,
 'openAccessPdf': {'url': '',
  'status': None,
  'license': None,
  'disclaimer': "Notice: This paper's abstract has been elided by the publisher. Paper

In [ ]:
def get_metadata(title: str):
    search_results = pyalex.Works().search(title).select(['id', 'doi', 'referenced_works']).get(page=1, per_page=1)
    return (search_results[0]['doi'], search_results[0]['id'], search_results[0]['referenced_works']) if search_results else (None, None, None)

df['title'] = df['fpath'].apply(get_title)

with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(tqdm(
        executor.map(get_metadata, df['title'].values),
        total=len(df)
    ))
df['doi', 'oaid', 'referenced_works'] = results

In [18]:
df.head()

,fpath,title,doi,oaid,referenced_works,total_references,references_in_dataset
0,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Fixed Point Diffusion Models,https://doi.org/10.1063/1.2121687,https://openalex.org/W2000456051,"[https://openalex.org/W1504980292, https://ope...",55,0
1,/cluster/home/lcarretero/workspace/dsl/dsl-res...,BEVNeXt: Reviving Dense BEV Frameworks for 3D ...,https://doi.org/10.1109/cvpr52733.2024.01901,https://openalex.org/W4402727763,"[https://openalex.org/W1861492603, https://ope...",73,0
2,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Leveraging Pre-trained Multi-task Deep Models ...,https://doi.org/10.1109/cvprw63382.2024.00473,https://openalex.org/W4402916217,[],0,0
3,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Insights from the Use of Previously Unseen Neu...,https://doi.org/10.48550/arxiv.2404.02189,https://openalex.org/W4393967825,[],0,0
4,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Efficient local correlation volume for unsuper...,https://doi.org/10.1109/cvprw63382.2024.00049,https://openalex.org/W4402904316,"[https://openalex.org/W1513100184, https://ope...",28,0
